In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 46.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=de8e270ed94c96df9a31b250468e1efe5f356effab1f40062d1d20263f245f3c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').\
        appName('jean').getOrCreate()
sc = spark.sparkContext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sc.addFile('https://www.dropbox.com/s/yviaftku1t09ejx/bible.txt?dl=1')

In [ ]:
# carregando o arquivo
rdd = sc.textFile('file://' + SparkFiles.get('bible.txt'))

In [ ]:
rdd.take(5)

['1:1 In the beginning God created the heaven and the earth.',
 '',
 '1:2 And the earth was without form, and void; and darkness was upon',
 'the face of the deep. And the Spirit of God moved upon the face of the',
 'waters.']

In [ ]:
# fazendo split de sentenças em palavras
palavras = rdd.flatMap(lambda frase: frase.split(' '))

In [ ]:
type(palavras)

pyspark.rdd.PipelinedRDD

In [ ]:
palavras.take(10)

['1:1',
 'In',
 'the',
 'beginning',
 'God',
 'created',
 'the',
 'heaven',
 'and',
 'the']

In [ ]:
palavras.count()

851195

In [ ]:
palavras.countByValue()

In [ ]:
# AULA 2 

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').\
        appName('jean').getOrCreate()

sc = spark.sparkContext

sc.addFile('https://www.dropbox.com/s/3k3v5tol6mypy35/airports.txt?dl=1')

rdd = sc.textFile("file://" + SparkFiles.get('airports.txt'))

linhas = rdd.map(lambda x: x.replace('\"', "").split(","))

# linhas.take(20)

# filtrando linhas dos USA
rdd_usa = rdd.filter(lambda l: l.split(',')[3] == '\"United States\"')

#rdd_usa.collect() #collect cria uma lista com todos os elementos

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd_amostra = rdd.sample(False, 0.1)
rdd_amostra.collect()

[]

In [ ]:
# media
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
soma = rdd.reduce(lambda x,y: x + y)
quantidade = rdd.count()

media = soma / quantidade

media

5.5

In [ ]:
# soma dos valores pares
somaPares = rdd.filter(lambda x: x % 2== 0).reduce(lambda x,y: x + y)
somaPares

30

In [ ]:
# atividade 1 - Média das latitudes dos Estados Uniddos 

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').\
        appName('jean').getOrCreate()

sc = spark.sparkContext

sc.addFile('https://www.dropbox.com/s/3k3v5tol6mypy35/airports.text?dl=1')

rdd = sc.textFile("file://" + SparkFiles.get('airports.txt'))

palavras = rdd.map(lambda x: x.replace('\"', "").split(","))

In [ ]:
# filter eh relacionado a linha inteira.
# map parte da linha (coluna).
palavras.filter(lambda x: x[3] == "United States")\
.map(lambda x: (x[3], float(x[6]))).groupByKey()\
.map(lambda x: (x[0], sum(list(x[1])) / len(list(x[1])))).collect()

[('United States', 41.444291588664015)]

In [ ]:
# Atividade 1 feita pelo professor

# 1 - Filtrar aeroportos nos USA
rdd_usa = rdd.filter(lambda x: \
                     x.split(',')[3] == '"United States"')
# 2 - calcular a latitude media destes aeroportos
rdd_latitudes = rdd_usa.map(lambda x: float(x.split(',')[6]))

#somar todas as latitudes
soma_latitudes = rdd_latitudes.reduce(lambda x, y: x + y) # primeiro comando de processamento de dados.
qtd_elementos = rdd_latitudes.count()

#media 
media = soma_latitudes / qtd_elementos
media                                

41.44429158866408

In [ ]:
sc.addFile('https://www.dropbox.com/s/yviaftku1t09ejx/bible.txt?dl=1')
rdd = sc.textFile("file://" + SparkFiles.get('bible.txt'))

In [ ]:
# Atividade 2:

# 1 - contar frequencia de palavras que se iniciam com vogais

rdd_palavras = rdd.flatMap(lambda x: x.split(' '))

rdd_palavras_semvazio = rdd_palavras.filter(lambda x: len(x)>0)
rdd_vogal = rdd_palavras_semvazio.filter(lambda x: x[0].lower() in 'aeiou')

rdd_vogal.count()

226451

In [ ]:
# Atividade 2:

# 2 quantas palavras iniciam com cada vogal

rdd_inicial_vogal = rdd_vogal.flatMap(lambda x: x[0].lower())
rdd_inicial_vogal.countByValue()

defaultdict(int, {'a': 98347, 'e': 13178, 'i': 44286, 'o': 52501, 'u': 18139})

In [ ]:
# Atividade 2:

# 3 contar a ocorrencia de cada volgal no texto
rdd_caracteres = rdd_palavras_semvazio.flatMap(lambda x: list(x))
rdd_caracteres_lower = rdd_caracteres.map(lambda x: x.lower())
rdd_caracteres_lower.filter(lambda x: x in "aeiou").countByValue()

defaultdict(int,
            {'a': 275701, 'e': 412160, 'i': 193926, 'o': 243143, 'u': 83462})

In [ ]:
# ===============================================
# AULA DIA 09/05/2022 


# montar um pair rdd no formato (cidade, pais) 
prdd = rdd.map(lambda x: (x.split(',')[2], x.split(',')[3]))
prdd.take(3)

[('"Goroka"', '"Papua New Guinea"'),
 ('"Madang"', '"Papua New Guinea"'),
 ('"Mount Hagen"', '"Papua New Guinea"')]

In [ ]:
# remover aspas duplas de chave e valor.

prdd_limpo = prdd.map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '')))
prdd_limpo.take(3)

[('Goroka', 'Papua New Guinea'),
 ('Madang', 'Papua New Guinea'),
 ('Mount Hagen', 'Papua New Guinea')]

In [ ]:
# mantendo apenas elementos dos estados unidos
prdd_usa = prdd_limpo.filter(lambda x: x[1] == 'United States')
prdd_usa.take(3)

[('Greencastle', 'United States'),
 ('Dowagiac', 'United States'),
 ('Cambridge', 'United States')]

In [ ]:
# convertendo paises para caixa alta (MAP)

prdd_usa_uc = prdd_usa.map(lambda x: (x[0], x[1].upper()))
prdd_usa_uc.take(3)

[('Greencastle', 'UNITED STATES'),
 ('Dowagiac', 'UNITED STATES'),
 ('Cambridge', 'UNITED STATES')]

In [ ]:
# Possibilidade me mexer apenas nos valores da tupla.
prdd_usa_uc = prdd_usa.mapValues(lambda x: x.upper())
prdd_usa_uc.take(3)

[('Greencastle', 'UNITED STATES'),
 ('Dowagiac', 'UNITED STATES'),
 ('Cambridge', 'UNITED STATES')]

In [ ]:
# função para salvar os resultado em disco.
# cada part eh pq foi feito um processamento em dois cores.
# caso queira em um unico arquivo. antes do save colocar .coalesce(1)

prdd_usa_uc.saveAsTextFile('prdd.txt')

In [ ]:
# ================================================================================================================
# contagem de palavras usando flatMap()

palavras = rdd.flatMap(lambda x: x.split(' '))
palavras.countByValue()

In [ ]:
# contagem de palavras usando reduceByKey() -> distribuido
# (palavra, 1)
prdd = palavras.map(lambda x: (x, 1))

# reduzir (somar) as ocorrencias de cada palavra 
prdd_ocorrencia = prdd.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False)
prdd_ocorrencia.take(3)

[('the', 62252), ('and', 38641), ('of', 34547)]

In [ ]:
# ======================== SPARK SQL ==========================================

from pyspark.sql.functions import col, max, min, avg

In [ ]:
df = spark.read.csv("file://" + SparkFiles.get('airports.txt'), inferSchema=True)

# Exemplo onde sua base ja tem header
# df_tde = spark.read.csv("file://" + SparkFiles.get('transactions_amostra.csv'), sep=";",header=True, inferSchema=True)

In [ ]:
df.show(5)

+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|_c0|                 _c1|         _c2|             _c3|_c4| _c5|      _c6|       _c7| _c8| _c9|_c10|                _c11|
+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|  1|              Goroka|      Goroka|Papua New Guinea|GKA|AYGA|-6.081689|145.391881|5282|10.0|   U|Pacific/Port_Moresby|
|  2|              Madang|      Madang|Papua New Guinea|MAG|AYMD|-5.207083|  145.7887|  20|10.0|   U|Pacific/Port_Moresby|
|  3|         Mount Hagen| Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789|144.295861|5388|10.0|   U|Pacific/Port_Moresby|
|  4|              Nadzab|      Nadzab|Papua New Guinea|LAE|AYNZ|-6.569828|146.726242| 239|10.0|   U|Pacific/Port_Moresby|
|  5|Port Moresby Jack...|Port Moresby|Papua New Guinea|POM|AYPY|-9.443383| 147.22005| 146|10.0|   U|Pacific/Port_Moresby|
+---+-----------

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



In [ ]:
# Exemplo criando View

df.createOrReplaceTempView('vw_aeroportos')

In [ ]:
query = 'SELECT _c2, _c3, _c6, _c7 FROM vw_aeroportos WHERE _c3 LIKE "United States"'
spark.sql(query).show(10)

+------------+-------------+----------+-----------+
|         _c2|          _c3|       _c6|        _c7|
+------------+-------------+----------+-----------+
| Greencastle|United States|39.6335556|-86.8138056|
|    Dowagiac|United States|41.9929342|-86.1280125|
|   Cambridge|United States|39.9750278|-81.5775833|
|Sturgeon Bay|United States|44.8436667|-87.4215556|
|Stewartstown|United States|39.7948244|-76.6471914|
|   Pendleton|United States|    45.695|-118.841389|
|      Tyonek|United States| 61.076667|-151.138056|
| Riverton WY|United States| 43.064167|-108.459722|
| Montrose CO|United States| 38.509794|-107.894242|
| Bolingbrook|United States|41.6959744|-88.1292306|
+------------+-------------+----------+-----------+
only showing top 10 rows



In [ ]:
# Quantidade de aeroportos por pais

# SELECT COUNT(*) FROM vw_aeroportos GROUP BY country

df.groupBy('_c3').count().orderBy(col('count').desc()).show(10)

+--------------+-----+
|           _c3|count|
+--------------+-----+
| United States| 1697|
|        Canada|  435|
|       Germany|  321|
|     Australia|  263|
|        Russia|  249|
|        France|  233|
|         China|  219|
|        Brazil|  213|
|United Kingdom|  210|
|         India|  140|
+--------------+-----+
only showing top 10 rows



In [ ]:
# latitude media dos aeroportos dos USA

# SELECT AVG(latitude) FROM view WHERE country = "United States"

df.filter(col('_c3').like('United States')).select(avg(col('_c6'))).show(10)

# OU 


df.filter('_c3 = "United States"').select(avg(col('_c6'))).show(10)

+------------------+
|          avg(_c6)|
+------------------+
|41.444291588664015|
+------------------+

+------------------+
|          avg(_c6)|
+------------------+
|41.444291588664015|
+------------------+



In [ ]:
df = df.withColumnRenamed('_c3', 'country')
df.show(10)

+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|_c0|                 _c1|         _c2|         country|_c4| _c5|      _c6|       _c7| _c8| _c9|_c10|                _c11|
+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|  1|              Goroka|      Goroka|Papua New Guinea|GKA|AYGA|-6.081689|145.391881|5282|10.0|   U|Pacific/Port_Moresby|
|  2|              Madang|      Madang|Papua New Guinea|MAG|AYMD|-5.207083|  145.7887|  20|10.0|   U|Pacific/Port_Moresby|
|  3|         Mount Hagen| Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789|144.295861|5388|10.0|   U|Pacific/Port_Moresby|
|  4|              Nadzab|      Nadzab|Papua New Guinea|LAE|AYNZ|-6.569828|146.726242| 239|10.0|   U|Pacific/Port_Moresby|
|  5|Port Moresby Jack...|Port Moresby|Papua New Guinea|POM|AYPY|-9.443383| 147.22005| 146|10.0|   U|Pacific/Port_Moresby|
|  6|          W